In [ ]:
import numpy as np
import pandas as pd
import os
from glob import glob
import cv2

In [ ]:
all_xray_df = pd.read_csv('../input/data/Data_Entry_2017.csv')
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('../input/data','images*', '*', '*.png'))}

In [ ]:
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)

In [ ]:
cleaned = all_xray_df.rename(columns={'Finding Labels': 'labels'})
cleaned = cleaned.set_index('Image Index').labels.str.split('|', expand=True).stack().reset_index(level=1, drop=True).to_frame('lables')
cleaned.head()

In [ ]:
all_xray_df.set_index('Image Index', inplace=True)
prepared_df = all_xray_df.merge(cleaned, left_index = True, right_index=True)


In [ ]:
prepared_df['lables'].unique()



In [ ]:
# #getting only pneumonia images
fibrosis_df = prepared_df[prepared_df['lables'] == 'Fibrosis']



In [ ]:
# #getting no finding images
normal_df = prepared_df[prepared_df['lables'] == 'No Finding']

fibrosis_df['lables'] = 'fibrosis'
normal_df['lables'] = 'normal'

In [ ]:
p_df_1 = pd.DataFrame()
p_df_1['filename'] = fibrosis_df['path']
p_df_1['labels'] = fibrosis_df['lables']

n_df_1 = pd.DataFrame()
n_df_1['filename'] = normal_df['path']
n_df_1['labels'] = normal_df['lables']

n_df_1 = n_df_1.sample(2000).reset_index(drop = True)

In [ ]:
l = []
for i in range(len(p_df_1)):
    l.append(i)
    
p_df_1['id'] = l

l = []
for i in range(len(n_df_1)):
    l.append(i)
    
n_df_1['id'] = l

p_df_1 = p_df_1.set_index('id')
n_df_1 = n_df_1.set_index('id')

In [ ]:
new_df = pd.concat([p_df_1, n_df_1]).reset_index(drop = True).sample(frac = 1.0).reset_index(drop = True)

In [ ]:
from PIL import Image
op = Image.open
from IPython.display import display


In [ ]:
n = len(fibrosis_df)
# display(op(fibrosis_df.iloc[0]['path']))
fibrosis_df.iloc[0]['path']

In [ ]:
# !rm *
!mkdir Fibrosis

In [ ]:
for i in range(n):
    x = op(fibrosis_df.iloc[i]['path'])
    x.save('/kaggle/working/Fibrosis/'+fibrosis_df.iloc[i]['path'].split('/')[-1])

In [ ]:
import shutil
shutil.make_archive("Fibimage", 'zip', '/kaggle/working/Fibrosis/')


In [66]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/Fibimage.zip')

/kaggle/working/Fibimage.zip